# IBM watsonx.ai

## 概述

这将帮助你快速开始使用 [Watsonx 文档压缩器](/docs/concepts/#document_compressors)。关于 Watsonx 文档压缩器所有功能和配置的详细文档，请访问 [API 参考](https://api.js.langchain.com/modules/_langchain_community.document_compressors_ibm.html)。

### 集成详情

| 类 | 包 | [PY 支持](https://python.langchain.com/docs/integrations/retrievers/ibm_watsonx_ranker/) | 包下载量 | 包最新版本 |
| :--- | :--- | :---: | :---: | :---: |
| [`WatsonxRerank`](https://api.js.langchain.com/classes/_langchain_community.document_compressors_ibm.WatsonxRerank.html) | [@langchain/community](https://www.npmjs.com/package/@langchain/community) |  ✅ | ![NPM - 下载量](https://img.shields.io/npm/dm/@langchain/community?style=flat-square&label=%20&) | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 准备工作

要访问 IBM WatsonxAI 模型，你需要创建一个 IBM watsonx.ai 账户，获取 API 密钥或其他类型的凭证，并安装 `@langchain/community` 集成包。

### 凭证

前往 [IBM Cloud](https://cloud.ibm.com/login) 注册 IBM watsonx.ai 并生成一个 API 密钥或提供如下所示的其他身份验证方式。

#### IAM 身份验证

```bash
export WATSONX_AI_AUTH_TYPE=iam
export WATSONX_AI_APIKEY=<YOUR-APIKEY>
```

#### Bearer token 身份验证

```bash
export WATSONX_AI_AUTH_TYPE=bearertoken
export WATSONX_AI_BEARER_TOKEN=<YOUR-BEARER-TOKEN>
```

#### IBM watsonx.ai 软件身份验证

```bash
export WATSONX_AI_AUTH_TYPE=cp4d
export WATSONX_AI_USERNAME=<YOUR_USERNAME>
export WATSONX_AI_PASSWORD=<YOUR_PASSWORD>
export WATSONX_AI_URL=<URL>
```

一旦将这些设置放置到你的环境变量中并初始化对象，身份验证将自动进行。

也可以通过将这些值作为参数传递给新实例来完成身份验证。

## IAM 身份验证

```typescript
import { WatsonxLLM } from "@langchain/community/llms/ibm";

const props = {
  version: "YYYY-MM-DD",
  serviceUrl: "<SERVICE_URL>",
  projectId: "<PROJECT_ID>",
  watsonxAIAuthType: "iam",
  watsonxAIApikey: "<YOUR-APIKEY>",
};
const instance = new WatsonxLLM(props);
```

## Bearer token 身份验证

```typescript
import { WatsonxLLM } from "@langchain/community/llms/ibm";

const props = {
  version: "YYYY-MM-DD",
  serviceUrl: "<SERVICE_URL>",
  projectId: "<PROJECT_ID>",
  watsonxAIAuthType: "bearertoken",
  watsonxAIBearerToken: "<YOUR-BEARERTOKEN>",
};
const instance = new WatsonxLLM(props);
```

### IBM watsonx.ai 软件身份验证

```typescript
import { WatsonxLLM } from "@langchain/community/llms/ibm";

const props = {
  version: "YYYY-MM-DD",
  serviceUrl: "<SERVICE_URL>",
  projectId: "<PROJECT_ID>",
  watsonxAIAuthType: "cp4d",
  watsonxAIUsername: "<YOUR-USERNAME>",
  watsonxAIPassword: "<YOUR-PASSWORD>",
  watsonxAIUrl: "<url>",
};
const instance = new WatsonxLLM(props);
```

如果你想从单个查询中获取自动化追踪，也可以取消下面 [LangSmith](https://docs.smith.langchain.com/) API 密钥的注释以进行设置：

```typescript
// process.env.LANGSMITH_API_KEY = "<YOUR API KEY HERE>";
// process.env.LANGSMITH_TRACING = "true";
```

### 安装

该文档压缩器位于 `@langchain/community` 包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core
</Npm2Yarn>
```

## 实例化

现在我们可以实例化我们的压缩器：

In [ ]:
import { WatsonxRerank } from "@langchain/community/document_compressors/ibm";

const watsonxRerank = new WatsonxRerank({
  version: "2024-05-31",
  serviceUrl: process.env.WATSONX_AI_SERVICE_URL,
  projectId: process.env.WATSONX_AI_PROJECT_ID,
  model: "cross-encoder/ms-marco-minilm-l-12-v2",
});

## 用法

首先，使用嵌入、文本分割器和向量存储设置一个基本的RAG（检索增强生成）数据摄入管道。我们将使用它来针对选定的查询检索并重新排序一些文档：

In [ ]:
import { readFileSync } from "node:fs";
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { WatsonxEmbeddings } from "@langchain/community/embeddings/ibm";
import { CharacterTextSplitter } from "@langchain/textsplitters";

const embeddings = new WatsonxEmbeddings({
 version: "YYYY-MM-DD",
 serviceUrl: process.env.API_URL,
 projectId: "<PROJECT_ID>",
 spaceId: "<SPACE_ID>",
 model: "ibm/slate-125m-english-rtrvr",
});

const textSplitter = new CharacterTextSplitter({
  chunkSize: 400,
  chunkOverlap: 0,
});
  
const query = "What did the president say about Ketanji Brown Jackson";
const text = readFileSync("state_of_the_union.txt", "utf8");

const docs = await textSplitter.createDocuments([text]);
const vectorStore = await MemoryVectorStore.fromDocuments(docs, embeddings);
const vectorStoreRetriever = vectorStore.asRetriever();

const result = await vectorStoreRetriever.invoke(query);
console.log(result);

[
  Document {
    pageContent: 'And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
    metadata: { loc: [Object] },
    id: undefined
  },
  Document {
    pageContent: 'I spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. \n' +
      '\n' +
      'I’ve worked on these issues a long time. \n' +
      '\n' +
      'I know what works: Investing in crime preventionand community police officers who’ll walk the beat, who’ll know the neighborhood, and who can restore trust and safety.',
    metadata: { loc: [Object] },
    id: undefined
  },
  Document {
    pageContent: 'We are the only nation on Earth that has always turned every crisis we have faced into an opportunity. \n' +
      '\n' +
      'The only nation that ca

将选中的文档传递以重新排序，并为每个文档接收特定分数

In [ ]:
import { WatsonxRerank } from "@langchain/community/document_compressors/ibm";

const reranker = new WatsonxRerank({
  version: "2024-05-31",
  serviceUrl: process.env.WATSONX_AI_SERVICE_URL,
  projectId: process.env.WATSONX_AI_PROJECT_ID,
  model: "cross-encoder/ms-marco-minilm-l-12-v2",
});
const compressed = await reranker.rerank(result, query);
console.log(compressed);

[
  { index: 0, relevanceScore: 0.726995587348938 },
  { index: 1, relevanceScore: 0.5758284330368042 },
  { index: 2, relevanceScore: 0.5479092597961426 },
  { index: 3, relevanceScore: 0.5468723773956299 }
]


或者你可以让文档随结果一起返回，为此请使用.compressDocuments()方法如下所示。

In [ ]:
const compressedWithResults = await reranker.compressDocuments(result, query);
console.log(compressedWithResults);

[
  Document {
    pageContent: 'And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
    metadata: { loc: [Object], relevanceScore: 0.726995587348938 },
    id: undefined
  },
  Document {
    pageContent: 'I spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. \n' +
      '\n' +
      'I’ve worked on these issues a long time. \n' +
      '\n' +
      'I know what works: Investing in crime preventionand community police officers who’ll walk the beat, who’ll know the neighborhood, and who can restore trust and safety.',
    metadata: { loc: [Object], relevanceScore: 0.5758284330368042 },
    id: undefined
  },
  Document {
    pageContent: 'We are the only nation on Earth that has always turned every crisis we have faced

## API 参考

有关 Watsonx 文档压缩器所有功能和配置的详细文档，请访问 [API 参考](https://api.js.langchain.com/modules/_langchain_community.document_compressors_ibm.html)。